In [8]:
import os

with open(os.path.join('data', 'joke', 'data')) as f:
    lines = f.readlines()

for line in lines:
    print(line)

"ID","Joke"

1,"What did the bartender say to the jumper cables? You better not try to start anything."

2,"Don't you hate jokes about German sausage? They're the wurst!"

3,"Two artists had an art contest... It ended in a draw"

4,"Why did the chicken cross the playground? To get to the other slide."

5,"What gun do you use to hunt a moose? A moosecut!"

6,"If life gives you melons, you might have dyslexia."

7,"Broken pencils... ...are pointless."

8,"What did one snowman say to the other snowman? 'Do you smell carrots?'"

9,"How many hipsters does it take to change a lightbulb? It's a really obscure number. You've probably never heard of it."

10,"Where do sick boats go? The dock!"

11,"I like my slaves like I like my coffee: Free."

12,"My girlfriend told me she was leaving me because I keep pretending to be a Transformer... I said, No, wait! I can change!"

13,"Old Chinese proverb: Man who not shower in 7 days makes one reek."

14,"What did the owner of a brownie factory say when 

In [9]:
joke_id, joke = tuple(line.split(',"'))

In [34]:
from torch.utils.data import DataLoader, random_split
from transformers import AutoTokenizer

from nlp import datasets
import importlib
importlib.reload(datasets)

<module 'nlp.datasets' from 'c:\\Users\\jnicolow\\Documents\\courses\\fall2024\\ICS-661\\llm_finetuning\\nlp\\datasets.py'>

In [35]:
importlib.reload(datasets)
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens({'pad_token': '[PAD]'}) # there is no default


batch_size = 2

dataset = datasets.JokesDataset(filepath=os.path.join('data', 'joke', 'data'), tokenizer=tokenizer)
train_ratio = 0.8
eval_ratio = 0.2
total_size = len(dataset)
train_size = int(total_size * train_ratio)
eval_size = total_size - train_size
train_dataset, eval_dataset = random_split(dataset, [train_size, eval_size])


train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=False)



print(f'there are {len(train_loader)*batch_size} training examples')

for batch in train_loader:
    print(tokenizer.decode(batch['input_ids'][0]))
    print(tokenizer.decode(batch['target_ids'][0]))
    break

there are 1296 training examples
Why is Ireland[PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PA

In [36]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))  # resize the model's token embeddings to accommodate the new padding token

Embedding(50258, 768)

In [ ]:
import torch
from torch.optim import AdamW

# Move model to GPU if available
device = torch.device('cuda:1') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Set up optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)  # Learning rate of 2e-5 is typical for fine-tuning

# Training loop
def train_model(model, train_loader, optimizer, device, num_epochs=3):
    model.train()  # Set model to training mode
    for epoch in range(num_epochs):
        total_loss = 0
        for batch in train_loader:
            # Get inputs from batch and move to device
            input_ids = batch['input_ids'].to(device)
            labels = input_ids.clone()  # For language modeling, labels are the same as input_ids

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass: get predictions and compute loss
            outputs = model(input_ids=input_ids, labels=labels)
            loss = outputs.loss  # Loss is already calculated internally for AutoModelForCausalLM
            total_loss += loss.item()

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

        # Print the average loss for the epoch
        avg_loss = total_loss / len(train_loader)
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}')

# Evaluation function (optional)
def evaluate_model(model, eval_loader, device):
    model.eval()  # Set model to evaluation mode
    total_loss = 0

    with torch.no_grad():
        for batch in eval_loader:
            input_ids = batch['input_ids'].to(device)
            labels = input_ids.clone()  # For language modeling, labels are the same as input_ids

            # Forward pass
            outputs = model(input_ids=input_ids, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

    avg_loss = total_loss / len(eval_loader)
    print(f'Evaluation Loss: {avg_loss:.4f}')

# Fine-tune the model
train_model(model, train_loader, optimizer, device, num_epochs=3)

# Optionally evaluate the model
evaluate_model(model, eval_loader, device)

# Save the fine-tuned model
model.save_pretrained('models/fine_tuned_gpt2')
tokenizer.save_pretrained('models/fine_tuned_gpt2')


c:\Users\jnicolow\AppData\Local\anaconda3\envs\nlp-env\lib\site-packages\transformers\models\gpt2\modeling_gpt2.py:545: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
